# Use the `frcmod` and `mol2` files to test that we can solvate each host-guest system with `paprika`

In [71]:
import glob as glob
import os as os
import subprocess as sp
import numpy as np

import parmed as pmd
from paprika.align import zalign, rotate_about_z, get_theta
from paprika.dummy import add_dummy, write_dummy_mol2, write_dummy_frcmod
from paprika.tleap import System

In [72]:
complexes = glob.glob('./OA-G*')

First, let's align the host along the *z* axis, so we will pull the guest towards +z. I'm going to align the vector connecting the center of mass of the oxygens on bottom ring of OA and the bridging oxygens on the top ring of OA.

Then, because the "square ring" at the top of octa acid is slightly twisted and rotated (also distorted into a bit of a rhombus itself) relative to the *x* and *y* axes, I'm going to apply a slight rotation, using the `O16-O17` vector. This helps during solvation by aligning the square opening of the host with the rectangular solvation box.

In [73]:
for complx in complexes:
    system = pmd.load_file(os.path.join(complx, 'AMBER', 'complex.pdb'))
    system = zalign(system, '@O10,O11,O32,O26,O9,O24,O25,O12', 
                              '@O16,O17,O19,O18,O15,O14,O20,O13')
    theta = get_theta(system, '@O16', '@O17', axis='x')
    system = rotate_about_z(system, theta=theta, save=True, 
                            filename=os.path.join(complx, 'AMBER', 'complex_aligned.pdb'))


<img src="complex_aligned.png">

Next, let's add dummy atoms. The origin of the coordinate system is at the center of mass of the bottom oxygens (shown above). This is already a bit behind the guest's *z* coordinate and roughly inline with its orientation, so this is a fine place for the first dummy atom.

In [74]:
for complx in complexes:
    hg = pmd.load_file(os.path.join(complx, 'AMBER', 'complex_aligned.pdb'), structure=True)
    hg = add_dummy(hg, residue_name='DM1', z=0.000)
    hg = add_dummy(hg, residue_name='DM2', z=-3.000)
    hg = add_dummy(hg, residue_name='DM2', y=2.200, z=-5.200)
    hg.save(os.path.join(complx, 'AMBER', 'complex_dummy.pdb'), overwrite=True)
    
    write_dummy_frcmod(path=os.path.join(complx, 'AMBER'), atom_type='Du')
    
    write_dummy_mol2(path=os.path.join(complx, 'AMBER'), residue_name='DM1', atom_type='Du', 
                     filename='dm1.mol2')
    write_dummy_mol2(path=os.path.join(complx, 'AMBER'), residue_name='DM2', atom_type='Du', 
                     filename='dm2.mol2')
    write_dummy_mol2(path=os.path.join(complx, 'AMBER'), residue_name='DM3', atom_type='Du', 
                     filename='dm3.mol2')

Next, let's solvate with 2500 waters like Jane did with this host during SAMPL5.

I pre-calculated that adding 4 Na and 4 Cl ions (in excessu to the 9 Na ions to neutralize the charge) gives the closest match to ~60 mM buffer conditions specified in the instructions.

In [75]:
for complx in complexes:
    system = System()
        
    system.output_path = os.path.join(complx, 'AMBER')
    system.target_waters = 2500
    system.output_prefix = 'solvate'
       
    system.neutralize = True
    system.pbc_type = 'rectangular'
    system.add_ions = ['Na+', 4, 'Cl-', 4]
    system.template_lines = ['source leaprc.water.tip3p',
                           'source leaprc.gaff',
                           'loadamberparams hst.frcmod',
                           'HST = loadmol2 hst.mol2',
                           'loadamberparams gst.frcmod',
                           'GST = loadmol2 gst.mol2',
                           'loadamberparams dummy.frcmod',
                           'DM1 = loadmol2 dm1.mol2',
                           'DM2 = loadmol2 dm2.mol2',
                           'DM3 = loadmol2 dm3.mol2',
                           'model = loadpdb complex_dummy.pdb'
    ]
    
    system.build()

In [76]:
for complx in sorted(complexes):
    print(complx)
    structure = pmd.load_file(os.path.join(complx, 'AMBER', 'solvate.prmtop'))
    
    # print(f'Total charge {pmd.tools.netCharge(structure).execute():0.3f} ...')
    
    sodium = len(structure[':Na+'].atoms)
    chloride = len(structure[':Cl-'].atoms)
        
    hst_charge = pmd.tools.netCharge(structure[':HST']).execute()
    gst_charge = pmd.tools.netCharge(structure[':GST']).execute()
    
    print(hst_charge)
    print(gst_charge)
    
    hst_gst_charge = hst_charge + gst_charge
    
    if hst_gst_charge < 0:
        sodium_added_to_neuralize = np.ceil(abs(hst_gst_charge))
        chloride_added_to_neutralize = 0
    elif hst_gst_charge > 0:
        sodium_added_to_neuralize = 0
        chloride_added_to_neutralize = np.ceil(abs(hst_gst_charge))
    else:
        sodium_added_to_neuralize = 0
        chloride_added_to_neutralize = 0
    
    print(f'{sodium_added_to_neuralize} Na+, {chloride_added_to_neutralize} Cl- for neutralization...')
    print(f'{sodium - sodium_added_to_neuralize} excess Na+, {chloride - chloride_added_to_neutralize} excess Cl-...') 

    
    counterion_mass = (sodium - sodium_added_to_neuralize) * 22.9898 + \
                      (chloride - chloride_added_to_neutralize) * 35.453
    
    AMU_TO_GRAMS = 1.66054 * 10**-24
    molar_concentration = counterion_mass * (AMU_TO_GRAMS) / (22.9898 + 35.453)

    volume = structure.get_box()[0] * structure.get_box()[1] * structure.get_box()[2]
    ANGSTROM_CUBED_TO_LITERS = 1 * 10 ** -27
    liters = volume * ANGSTROM_CUBED_TO_LITERS
    
    moles_per_liter = molar_concentration / liters
    print(f'Counterion molarity {moles_per_liter:0.3f} (beyond neutralization)...')

./OA-G3-0
-7.999999972451344
-1.0000000061463155
9.0 Na+, 0 Cl- for neutralization...
4.0 excess Na+, 4 excess Cl-...
Counterion molarity 0.062 (beyond neutralization)...
./OA-G3-1
-7.999999972451344
-1.0000000061463155
9.0 Na+, 0 Cl- for neutralization...
4.0 excess Na+, 4 excess Cl-...
Counterion molarity 0.062 (beyond neutralization)...
./OA-G3-2
-7.999999972451344
-1.0000000061463155
9.0 Na+, 0 Cl- for neutralization...
4.0 excess Na+, 4 excess Cl-...
Counterion molarity 0.062 (beyond neutralization)...
./OA-G3-3
-7.999999972451344
-1.0000000061463155
9.0 Na+, 0 Cl- for neutralization...
4.0 excess Na+, 4 excess Cl-...
Counterion molarity 0.063 (beyond neutralization)...
./OA-G3-4
-7.999999972451344
-1.0000000061463155
9.0 Na+, 0 Cl- for neutralization...
4.0 excess Na+, 4 excess Cl-...
Counterion molarity 0.062 (beyond neutralization)...
./OA-G6-0
-7.999999972451344
-1.0000000029085245
9.0 Na+, 0 Cl- for neutralization...
4.0 excess Na+, 4 excess Cl-...
Counterion molarity 0.062 (